<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br><br><br>

# Listed Volatility and Variance Derivatives

**Wiley Finance (2017)**

Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [@dyjh](http://twitter.com/dyjh) | http://books.tpq.io

<img src="https://hilpisch.com/images/lvvd_cover.png" alt="Listed Volatility and Variance Derivatives" width="30%" align="left" border="0">

# Variance Futures at Eurex

## Introduction

Based on the previous chapter, this chapter introduces the Eurex variance futures contracts and their major characteristics. It covers mainly the following topics:

* introduction and motivation to and for Eurex variance futures
* variance futures concepts needed to understand and trade in Eurex variance futures
* example calculation for a variance future illustrating the concepts by numerical results
* comparison between variance swaps and futures based on the numerical example

By introducing variance futures on the EURO STOXX 50 stock index, Eurex standardizes one of the most popular types of volatility/variance derivatives, namely variance swaps. Such a standardization brings a number of benefits to the markets:

* **unified terms**: standardization leads to unified, well-documentad terms
* **transparency**: centralized trading at Eurex increases transparency for all market participants
* **liquidity**: standardization and centralization increases liquidity (market depth) in the variance futures
* **fungibility**: variance futures are fully fungible and can be traded in and out at any point over their life time
* **increased market integrity**: clearing by Eurex ensures, among others, low counterparty risk and high risk management standards

Variance futures replicate the payoff of Over-the-Counter (OTC) variance swaps and are traded based on OTC conventions in vega notional and at volatility strikes. Daily margin payments based on settlement prices add up to the final settlement payoff of the OTC variance swap. Trading start for the variance futures of Eurex has been the 22. September 2014.

Net present value (NPV) effects are accounted for by two different pricing components:

* discount factor as for the mark-to-market for variance swaps
* accumulated return on modified variation margin (ARMVM) for the daily margin payments

In order to perfectly mimic the cash flow of a variance swap with the same terms, variance futures trading makes certain conversions necessary. This is mainly due to the variance swaps being based on end-of-day returns of the respective index (which cannot be exactly known during the day). Therefore, there are two conversions:

* **intraday**: intraday trades are booked with the desired quantity and at preliminary futures prices
* **end-of-day**: at the end of the day, intraday trades are cancelled and re-booked with the same quantity at the futures settlement prices (taking into account the realized variance for the rest of the day, i.e. since the time the trade was initially booked)

All trades are booked and conversions are made by the Eurex T7 trading system. While variance futures are quoted in Vega notional and volatility strikes, bookings and clearings are made only in futures and futures prices. 

## Variance Futures Concepts

Standardized trading and settlement in variance futures is based on a number of financial concepts. This section introduces all concepts necessary and provides formal definitions.

### Realized Variance

Realized variance $\sigma^2$ for the Eurex variance futures is defined as

$$\sigma^2 \equiv 10000 \cdot \frac{252}{N} \cdot \sum^{N}_{n=1}R_n^2$$

where, for a time series $S_n, n=0, 1, ..., N$, of daily EURO STOXX 50 closing values, the log returns are given by

$$R_n \equiv \log \frac{S_{n}}{S_{n-1}}$$

Here, it is assumed that there are 252 trading days per year and that the average daily return is zero. Note the scaling factor of 10,000 to get to the market convention of quoting variance in percent and not in decimal values.

### Net Present Value Concepts

There are two concepts that take into account that the daily margin cash flows of trading futures leads, time-wise, to a different cash flow profile than the single payoff of a variance future at maturity. 

#### Discount Factor

The first is the discount factor $DF_t$ which is defined for a maturity date $T$ and a current date $t$ by

$$DF_t = e^{-\frac{r (T-t)}{365}}$$

Here, $r$ is the relevant interest rate to be applied to the remaining time-to-maturity $(T - t)$, calculated in days. Eurex uses Euribor rates to calculate the discount factor.

Let us have a look at the development of the Euribor rates in 2014 from January to the beginning of September. Euribor rates are available on the web site http://www.euribor-rates.eu or a bit more convenient as Excel spreadsheet files for download on the web site http://www.emmi-benchmarks.eu/euribor-org/euribor-rates.html. From this last web site, we have downloaded Euribor rates data for 2014 which we use in the following. For details on the Python usage that follows, see chapter _Python Introduction_ or refer to Hilpisch (2018): _Python for Finance_. pandas is again the library we are working with.

In [ ]:
import pandas as pd

With pandas, you can read data from a spreadsheet file as follows.

In [ ]:
## read data from CSV file
eb = pd.read_csv('./data/hist_EURIBOR_2015.csv',  # filename
                   index_col=0,  # index column
                   parse_dates=True,  # parsing date information
                   dayfirst=True)  # European date convention
eb.info()

The following figure shows the evolution of the eight different rates over time (values in percent). 

In [ ]:
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
eb.plot(figsize=(10, 6));

<p style="font-family: monospace;">Historical Euribor rates.

Short term interest rates are pretty low, leading to a rather small impact of the discount factor in such an environment. Let us calculate the discount factor for the longest maturity with the last available rate for the 12 months horizon.

In [ ]:
eb.tail()

For a 12 months period the discount effect is less than half a percent.

In [ ]:
import math
math.exp(-0.00168 * 365. / 365)

#### ARMVM

The accumulated return on modified variation margin (ARMVM) takes into account that variance futures trading leads to daily margin payments that have an impact on the present value of the variance future. For given dates $t$ and $t-1$, ARMVM is given by 

$$ARMVM_t = ARMVM_{t-1}\cdot e^{\frac{r\Delta t}{365}} + \left(F_{t-1}^S - C \right)\left(e^{\frac{r\Delta t}{365}}-1\right)$$


Here, $r$ is the relevant interest rate to be used, $\Delta t$ is the difference between dates $t$ and $t-1$ in days, $F_{t-1}^S$ is the variance futures settlement price at $t-1$ and $C$ is a constant fixed at a level of 3,000. If $t=1$, i.e. on the first settlement day (say Monday), $ARMVM_{1} \equiv 0$ and $F_{1}^S \equiv C$.

The relevant rate for the ARMVM calculation is the Eonia rate. On the web site http://www.emmi-benchmarks.eu/euribor-eonia-org/about-eonia.html you find the following explanation:

> "Eonia® (Euro OverNight Index Average) is the effective overnight reference rate for the euro. It is computed as a weighted average of all overnight unsecured lending transactions in the interbank market, undertaken in the European Union and European Free Trade Association (EFTA) countries."

This web site also provides historical Eonia data for download. Let us have a look at the historical development of the Eonia rates for a couple of weeks in 2015.

In [ ]:
## read data from Excel spreadsheet file
eo = pd.read_csv('./data/hist_EONIA_2015.csv',  # filename
                   index_col=0,  # index column
                   parse_dates=True,  # parsing date information
                   dayfirst=True)  # European date convention
eo.info()

Again, we can easily visualize the data set (see the following figure).

In [ ]:
eo.plot(figsize=(10, 5));

<p style="font-family: monospace;">Historical Eonia rates.

The last available values in the data set for Eonia are:

In [ ]:
eo.tail()

Let us work with a value (in decimals) of $r_{eo} = -0.001$.

In [ ]:
r_eo = -0.001

Now consider the second settlement day $t=2$ (say Tuesday), assume that a rate of $r_{eo}$ applies and remember that $F_{1}^S=3000$. We then have

$$
ARMVM_2 = 0 \cdot e^{\frac{r_{eo} \cdot 1}{365}} + \left(3000 - 3000 \right)\left(e^{\frac{r_{eo} \cdot 1}{365}}-1\right) = 0
$$

Consider now the third settlement day $t=3$ (say Wednesday). Assume, that the futures settlement price has risen the day before to $F_2^S = 3100$. Now

$$
ARMVM_3 = 0 \cdot e^{\frac{r_{eo} \cdot 1}{365}} + \left(3100 - 3000 \right)\left(e^{\frac{r_{eo} \cdot 1}{365}}-1\right)
$$

The effect is rather small.

In [ ]:
ARMVM3 = 0 * math.exp(r_eo / 365) + (3100 - 3000) * (math.exp(r_eo / 365) - 1)
ARMVM3

One day later, on the fourth settlement day $t=4$ and assuming that the variance futures settlement price has fallen the day before to $F_3 = 3050$ we get

$$
ARMVM_4 = ARMVM_3 \cdot e^{\frac{r_{eo} \cdot 1}{365}} + \left(3050 - 3000 \right)\left(e^{\frac{r_{eo} \cdot 1}{365}}-1\right)
$$

In [ ]:
ARMVM4 = (ARMVM3 * math.exp(r_eo / 365)
        + (3050 - 3000) * (math.exp(r_eo / 365) - 1))
ARMVM4

### Traded Variance Strike

The traded variance strike $\sigma_t^2$ at date $t$ is the time weighted average of the realized variance $\sigma_{0,t}^2$ weighted by the number of elapsed days $t$ and the traded implied variance $\sigma_{i}^2(t,T)$ weighted by the remaining life time of the variance future in days:

$$
\sigma_t^2 = \frac{t \cdot \sigma_{0,t}^2 + (T - t) \cdot \sigma_{i}^2(t,T)}{T}
$$

### Traded Futures Price

The traded futures price $F_t$ given the other quantities from before is then defined as

$$
F_t = DF_t \cdot \left(\sigma_t^2 -\sigma_{i}^2(t,T) \right) - ARMVM_t + 3000
$$

The scaling constant $C=3000$ is chosen to ensure that the futures price cannot become negative.

### Number of Futures

Variance futures are traded in Vega notional and volatility strikes. However, settlement takes place in futures (or variance notional) and variance strikes. The number of futures is calculated according to the formula

$$
futures = \frac{VegaNotional}{2 \cdot \sigma_{i}(t,T)} \cdot \frac{T}{T-t}
$$

with $\sigma_{i}(t,T)$ as the traded implied volatility strike.

### Par Variance Strike

At daily settlement, i.e. when the log return for the EURO STOXX 50 for that day is known, the par variance strike $\sigma_p^2$ is calculated using the then current, settled implied volatility $\sigma_i$. 

$$
\sigma_p^2 = \frac{t \cdot \sigma_{0,t}^2 + (T-t) \cdot \sigma_i^2}{T}
$$



### Futures Settlement Price

Finally, the daily futures settlement price $F_t^S$ is calculated according to the following formula where all components are as defined as before.

$$
F_t^S = DF_t \cdot \left(\sigma_p^2 - \sigma_K^2\right) - ARMVM_t + 3000
$$

The futures settlement price is the mark-to-market value of the corresponding variance swap minus the ARMVM plus the scaling factor of 3,000.

## Example Calculation for a Variance Future

In this section, we conduct an example calculation for the Eurex variance futures contract given historical data as used in section _Variance Swaps_.

In [ ]:
import pandas as pd
h5 = pd.HDFStore('./data/SX5E_V6I1.h5', 'r')
data = h5['SX5E_V6I1']
h5.close()
data

We add Euribor and Eonia data to the pandas ``DataFrame`` object. For simplicity, we use the 2 week Euribor values throughout. 

In [ ]:
data = data.join(eb['2w'], how='left')
data = data.join(eo, how='left')
data

Let us add the log returns to the data set as well as the realized variance.

In [ ]:
import numpy as np
data['R_t'] = np.log(data['SX5E'] / data['SX5E'].shift(1))
data['sigma**2'] = 10000 * 252 * (np.cumsum(data['R_t'] ** 2)
                                  / np.arange(len(data)))
data

Assume that the variance future comes to life on 01. June 2015 and that it matures on 19. June 2015. This is a maturity of $T=15$ trading days. Let us generate in addition an array with all (elapsed) trading days over time.

In [ ]:
T = 15.
data['t'] = np.arange(1, 16)
data['t']

Assuming a constant Euribor rate of -0.1%, we can add a new column to the ``DataFrame`` object with the discount factors in vectorized fashion (see chapter _Trading and Settlement_ for further details on the discount factor calculation).

In [ ]:
r_eb = -0.001
data['DF_t'] = np.exp(-r_eb * (T - data['t']) / 365.) 
data

The standard volatility strike is $\sigma_K = 25.871$ and the standard variance strike is $\sigma_K^2=25.871^2=669.31$.

In [ ]:
sigma_K = data['V6I1'][0]
sigma_K

Assume a vega notional of 100,000. This translates into a variance notional of:

In [ ]:
Notional = 100000 / (2 * sigma_K)
Notional

The settlement price on the first trading day is standardized to 3,000. We generate a new column in the pandas ``DataFrame`` object and initialize the first value (and all the others) accordingly.

In [ ]:
data['F_tS'] = 3000

data

The ARMVM on the first settlement day is zero. We again generate a new column and initialize all values with zero.

In [ ]:
data['ARMVM_t'] = 0.0

The futures price on the second settlement day is given as:

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
data['F_tS'][1] = data['DF_t'][1] * ((data['t'][1] * data['sigma**2'][1]
                    + (T - data['t'][1]) * data['V6I1'][1] ** 2) / T
                    - sigma_K ** 2) + 3000
data['F_tS'][1]                       

Analogously, we can calculate the settlement prices for all other settlement days. Note that we take the index level of the EURO STOXX 50 at close and not the average of its level between 11:50 and 12:00 CET at the final settlement day (see chapter _Trading and Settlement_).

In [ ]:
for t in data.index[1:]:
    data['ARMVM_t'][t] = (data['ARMVM_t'].shift(1)[t]
                          * math.exp(data['EONIA'].shift(1)[t] / 252)
                       + (data['F_tS'].shift(1)[t] - 3000)
                         * (math.exp(data['EONIA'].shift(1)[t] / 252) - 1))
    data['F_tS'][t] = data['DF_t'][t] * ((data['t'][t] * data['sigma**2'][t]
                          + (T - data['t'][t]) * data['V6I1'][t] ** 2) / T
                          - sigma_K ** 2) - data['ARMVM_t'][t] + 3000

We end up with a complete data set, and in particular the simulated variance futures settlement values. 

In [ ]:
data

The following gifure shows the results graphically.

In [ ]:
data[['SX5E', 'sigma**2', 'F_tS']].plot(subplots=True,
                                       #color='blue',
                                       figsize=(10, 9));

<p style="font-family: monospace;">Calculated variance futures settlement values for the EURO STOXX 50.

We save the generated data set for re-use in the next section.

In [ ]:
h5 = pd.HDFStore('./data/var_data.h5', 'a')
h5['var_future'] = data
h5.close()

## Comparison of Variance Swap and Future

Eurex variance futures are a means to replicate the payoff of OTC-traded variance swaps with a listed, standardized product. Let us compare the mark-to-market values of the variance swap from section _Variandce Swaps_ with the settlement values calculated in the previous section.

To this end, we import the data for the variance swap first.

In [ ]:
h5 = pd.HDFStore('./data/var_data.h5', 'r')
var_swap = h5['var_swap']
h5.close()

Next, we combine and plot the two time series for the variance swap and the variance future, respectively, against each other. Note that we subtract the constant $C$ for the comparison.

In [ ]:
comp = pd.DataFrame({'F_tS': data['F_tS'] - 3000,
                     'V_t': var_swap['V_t']}, index=data.index)
comp

The following figure compares the two time series.

In [ ]:
comp.plot(style=['b', 'ro'], figsize=(9, 5));

<p style="font-family: monospace;">Variance swap and futures prices for EURO STOXX 50 compared.

Finally, the following figure presents the absolute differences which are quite small.

In [ ]:
(comp['F_tS'] - comp['V_t']).plot(style='r^', figsize=(9, 5));

<p style="font-family: monospace;">Differences between variance swap and futures prices.

## Conclusions

This chapter deals with the basic notions and concepts that need to be understood to trade Eurex variance futures based on the EURO STOXX 50 stock index. It introduces all concepts in formal fashion and illustrates their calculation based on a concrete numerical example which is based on the same data as the variance swap example calculation in the previous chapter. This makes it possible to numerically show that indeed the construction of the Eurex variance futures contract replicates the payoff of typical OTC variance contracts while adding intraday liquidity and tradeability. The subsequent chapter covers special aspects that relate to the intraday trading and end-of-day settlement of these derivatives products.

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>